In [ ]:
# ✅ 설치 & 런타임 자동 재시작
!pip install --quiet \
  diffusers==0.26.3 \
  transformers==4.39.3 \
  huggingface_hub==0.25.1 \
  accelerate==0.29.2 \
  gradio==3.50.2 \
  peft==0.10.0 \
  safetensors torch torchvision torchaudio numpy scipy

import os, IPython
print("✅ 설치 완료. 런타임을 자동으로 재시작합니다.")
os.kill(os.getpid(), 9)


In [ ]:
# ✅ Hugging Face 로그인
from huggingface_hub import login
login()


In [ ]:
# ✅ 모델 로딩
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
import torch

pipe_txt2img = StableDiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    torch_dtype=torch.float16
).to("cuda")

pipe_img2img = StableDiffusionImg2ImgPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    torch_dtype=torch.float16
).to("cuda")


In [ ]:
# ✅ 이미지 생성 함수
from PIL import Image
import numpy as np

def infer(prompt, init_image=None, strength=0.75):
    if init_image is not None:
        if isinstance(init_image, np.ndarray):
            init_image = Image.fromarray(init_image)
        elif not isinstance(init_image, Image.Image):
            raise ValueError(f"Unsupported image type: {type(init_image)}")
        init_image = init_image.resize((512, 512))

        with torch.autocast("cuda"):
            result = pipe_img2img(
                prompt=[prompt],
                image=init_image,
                strength=strength,
                guidance_scale=7.5
            )
    else:
        with torch.autocast("cuda"):
            result = pipe_txt2img(
                prompt=[prompt],
                guidance_scale=7.5
            )
    return result.images


In [ ]:
# ✅ Gradio 인터페이스 구성
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("# 🎨 Stable Diffusion Image Generator")

    prompt_input = gr.Textbox(label="프롬프트", value="a cushion on a car seat")
    init_image = gr.Image(label="초기 이미지 (선택)", type="numpy")
    strength = gr.Slider(label="유지 강도", minimum=0.0, maximum=1.0, value=0.75)
    gallery = gr.Gallery(label="결과 이미지")

    run_button = gr.Button("이미지 생성")

    run_button.click(fn=infer, inputs=[prompt_input, init_image, strength], outputs=[gallery])

demo.launch(debug=True, share=True)
